In [1]:
import pandas as pd
import numpy as np
import rolling_hrp as rol
import HRP

df = pd.read_excel("data/ibrx_mensal.xlsx", na_values="-")

# transforming the 'Data' column to a date format
df['Data'] = df['Data'].str.replace('Jan', '01').str.replace('Fev', '02').str.replace('Mar', '03').str.replace('Abr', '04').str.replace('Mai', '05').str.replace('Jun', '06').str.replace('Jul', '07').str.replace('Ago', '08').str.replace('Set', '09').str.replace('Out', '10').str.replace('Nov', '11').str.replace('Dez', '12')

# filtering based on date
def filter_covid(df, covid = True):

    ''' If covid is set to true (default), the return will be a dataframe including the covid window.'''

    if covid == True:
        df['Data'] = pd.to_datetime(df['Data'], format='%m-%Y')
        df = df[(df['Data'] >= '2000-01-01') & (df['Data'] <= '2022-06-01')]
        df = df.dropna(axis=1)
        return df
    else:
        df['Data'] = pd.to_datetime(df['Data'], format='%m-%Y')
        df = df[(df['Data'] >= '2000-01-01') & (df['Data'] <= '2020-02-01')]
        df = df.dropna(axis=1)
        return df

df_covid = filter_covid(df, True)
df_w_covid = filter_covid(df, False)

In [5]:
df_w_covid_no_date = df_w_covid.iloc[:, 1:] # skipping the 'Data' column

results_60 = rol.rolling_window_HRP(df_w_covid_no_date, 60)
results_60 = results_60.reset_index(drop=True)
results_60['Date'] = df_w_covid['Data'].reset_index(drop=True)

results_60

c:\Users\felip\OneDrive\Área de Trabalho\python\BDAQ\HRP.py:19: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  clustering = linkage(euclidean_distance, method=linkage_method)


,Date,Equal Weights,HRP,MV
0,2000-01-01,-5.999268,-4.994623,-8.396179
1,2000-02-01,15.019031,13.874689,14.817926
2,2000-03-01,-6.380284,-7.403602,-5.429452
3,2000-04-01,-6.549963,-9.712360,-19.441430
4,2000-05-01,3.881629,2.924667,4.257075
...,...,...,...,...
177,2014-10-01,1.006296,-0.991657,-3.551084
178,2014-11-01,3.352928,3.715713,3.305966
179,2014-12-01,9.734312,8.935761,5.657007
180,2015-01-01,-1.348356,-0.539980,5.627517


In [12]:
results_120 = rol.rolling_window_HRP(df2, 120)

results_120 = results_120.reset_index(drop=True)
results_120['Date'] = df['Data'].reset_index(drop=True)

results_120

c:\Users\felip\OneDrive\Área de Trabalho\python\BDAQ\HRP.py:19: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  clustering = linkage(euclidean_distance, method=linkage_method)


,Date,Equal Weights,HRP,MV
0,2020-02-01,-0.228344,-0.579897,0.474058
1,2020-01-01,-2.882625,-2.131993,-3.871228
2,2019-12-01,5.906200,5.569836,3.389351
3,2019-11-01,6.712898,4.845041,5.167806
4,2019-10-01,-0.969631,-0.518860,1.161964
...,...,...,...,...
117,2010-05-01,-0.663264,-0.397395,2.530252
118,2010-04-01,-7.448902,-7.384759,-5.213107
119,2010-03-01,1.999222,2.308414,3.600595
120,2010-02-01,0.368432,0.246268,1.006072


In [13]:
stats60 = results_60.describe(percentiles=[0.25, 0.5, 0.75])
stats60.loc['std'] = results_60.std()  # Add standard deviation
stats60.loc['skewness'] = results_60.skew()  # Add skewness
stats60.loc['kurtosis'] = results_60.kurtosis()  # Add kurtosis

stats60

C:\Users\felip\AppData\Local\Temp\ipykernel_4156\2279686666.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  stats60.loc['skewness'] = results_60.skew()  # Add skewness
C:\Users\felip\AppData\Local\Temp\ipykernel_4156\2279686666.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  stats60.loc['kurtosis'] = results_60.kurtosis()  # Add kurtosis


,Equal Weights,HRP,MV
count,182.000000,182.000000,182.000000
mean,1.617621,1.966993,1.606141
std,6.879766,6.061408,6.815061
min,-19.326889,-15.994539,-15.857643
25%,-2.764310,-1.536641,-1.819389
50%,1.467849,1.481105,0.861471
75%,6.163281,5.642339,5.019228
max,18.517063,20.538999,22.970048
skewness,-0.044152,0.106295,0.339303
kurtosis,0.245945,0.643967,0.704210


In [14]:
stats120 = results_120.describe(percentiles=[0.25, 0.5, 0.75])
stats120.loc['std'] = results_60.std()  # Add standard deviation
stats120.loc['skewness'] = results_60.skew()  # Add skewness
stats120.loc['kurtosis'] = results_60.kurtosis()  # Add kurtosis

stats120

C:\Users\felip\AppData\Local\Temp\ipykernel_4156\4147853140.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  stats120.loc['skewness'] = results_60.skew()  # Add skewness
C:\Users\felip\AppData\Local\Temp\ipykernel_4156\4147853140.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  stats120.loc['kurtosis'] = results_60.kurtosis()  # Add kurtosis


,Equal Weights,HRP,MV
count,122.000000,122.000000,122.000000
mean,2.293516,2.114847,2.248037
std,6.879766,6.061408,6.815061
min,-19.326889,-16.181627,-18.866315
25%,-2.920641,-2.254478,-2.164824
50%,2.424704,2.252456,2.135890
75%,6.794287,6.445362,6.197744
max,18.517063,18.518277,21.880523
skewness,-0.044152,0.106295,0.339303
kurtosis,0.245945,0.643967,0.704210


In [16]:

# concatenate the dataframes horizontally
stats_combined = pd.concat([stats120, stats60], axis=1)

# rename the columns to include the strategy name
stats_combined.columns = ['Equal Weights 120', 'HRP 120', 'MV 120', 'Equal Weights 60', 'HRP 60', 'MV 60']

# drop the duplicate 'Period' column
stats_combined = stats_combined.loc[:, ~stats_combined.columns.duplicated()]

stats_combined

,Equal Weights 120,HRP 120,MV 120,Equal Weights 60,HRP 60,MV 60
count,122.000000,122.000000,122.000000,182.000000,182.000000,182.000000
mean,2.293516,2.114847,2.248037,1.617621,1.966993,1.606141
std,6.879766,6.061408,6.815061,6.879766,6.061408,6.815061
min,-19.326889,-16.181627,-18.866315,-19.326889,-15.994539,-15.857643
25%,-2.920641,-2.254478,-2.164824,-2.764310,-1.536641,-1.819389
50%,2.424704,2.252456,2.135890,1.467849,1.481105,0.861471
75%,6.794287,6.445362,6.197744,6.163281,5.642339,5.019228
max,18.517063,18.518277,21.880523,18.517063,20.538999,22.970048
skewness,-0.044152,0.106295,0.339303,-0.044152,0.106295,0.339303
kurtosis,0.245945,0.643967,0.704210,0.245945,0.643967,0.704210


In [30]:
corr_matrix = df2.corr()
corr_matrix.to_csv('data\correlation_matrix.csv')

In [27]:
results = pd.DataFrame(index=corr_matrix.index, columns=['min', 'max', 'mean', 'sd'])

for asset in corr_matrix.columns:
    results.loc[asset, 'min'] = corr_matrix[asset].min()
    results.loc[asset, 'max'] = corr_matrix[asset].max()
    results.loc[asset, 'mean'] = corr_matrix[asset].mean()
    results.loc[asset, 'sd'] = corr_matrix[asset].std()

results

,min,max,mean,sd
ALPA4,0.170371,1.0,0.309944,0.156106
ABEV3,0.146747,1.0,0.264593,0.163197
BBDC3,0.212054,1.0,0.477464,0.214255
BBDC4,0.207087,1.0,0.504095,0.221847
BBAS3,0.20087,1.0,0.501014,0.200654
BRKM5,0.121283,1.0,0.274911,0.157905
CMIG4,0.130151,1.0,0.431512,0.184449
CPLE6,0.165105,1.0,0.421973,0.175769
ELET3,0.085753,1.0,0.403645,0.210201
ELET6,0.121475,1.0,0.402664,0.206967
